## Json 바탕 이상데이터 생성


In [3]:
import json, numpy as np, open3d as o3d, os

# ---------- 1. JSON 파라미터 ----------
with open("cylinder_fitting_result.json") as f:
    info = json.load(f)
d  = np.asarray(info["params"][0])
p0 = np.asarray(info["params"][1])
r  = float(info["params"][2])
d /= np.linalg.norm(d)

# ---------- 2. half_h ----------
scan_xyz = np.loadtxt("cylinder.xyz")[:, :3]
proj = (scan_xyz - p0) @ d
half_h = (proj.max() - proj.min()) / 2

# ---------- 3. 균일 샘플 함수 ----------
def sample_cylinder_area_uniform(center, d, r, half_h,
                                 total_pts=15_000, add_caps=True):
    tmp = np.array([1,0,0]) if abs(d[0]) < 0.9 else np.array([0,1,0])
    u = np.cross(d, tmp);  u /= np.linalg.norm(u)
    v = np.cross(d, u)

    h = 2*half_h
    A_side = 2*np.pi*r*h
    A_cap  = np.pi*r*r
    if add_caps:
        A_tot = A_side + 2*A_cap
        n_side = int(total_pts * A_side / A_tot)
        n_cap  = int(total_pts * A_cap  / A_tot)
    else:
        n_side, n_cap = total_pts, 0

    # --- 측면 ---
    z_rand = np.random.uniform(-half_h, half_h, n_side)
    t_rand = np.random.uniform(0, 2*np.pi, n_side)
    side = (center
            + np.outer(z_rand, d)
            + np.outer(r*np.cos(t_rand), u)
            + np.outer(r*np.sin(t_rand), v))

    # --- 뚜껑 ---
    def disk(center_cap):
        rr = r*np.sqrt(np.random.rand(n_cap))
        tt = 2*np.pi*np.random.rand(n_cap)
        return (center_cap
                + np.outer(rr*np.cos(tt), u)
                + np.outer(rr*np.sin(tt), v))

    caps = []
    if add_caps and n_cap > 0:
        caps.append(disk(center + d*half_h))   # 윗뚜껑
        caps.append(disk(center - d*half_h))   # 아랫뚜껑

    return side, caps  # side:(n_side,3)  caps:[top,bottom]

# ---------- 4. 샘플링 ----------
side_pts, [top_pts, bot_pts] = sample_cylinder_area_uniform(
    p0, d, r, half_h, total_pts=15_000, add_caps=True)

# ---------- 5. 라벨링 ----------
# 0 = 아랫뚜껑, 1 = 윗뚜껑, 2 = 측면
lab_side = np.full(len(side_pts), 2, dtype=int)
lab_top  = np.full(len(top_pts),  1, dtype=int)
lab_bot  = np.full(len(bot_pts),  0, dtype=int)

pts = np.vstack([bot_pts, top_pts, side_pts])
labs = np.concatenate([lab_bot, lab_top, lab_side])

# ---------- 6. 저장 ----------
os.makedirs("data", exist_ok=True)
out_path = "./data/cylinder_synthetic_3cls.xyzc"
np.savetxt(out_path,
           np.hstack([pts, labs[:,None]]),
           fmt="%.6f %.6f %.6f %.8e")   # 라벨도 e‑표기

print("✔ 저장 완료 →", out_path)


✔ 저장 완료 → ./data/cylinder_synthetic_3cls.xyzc


## 오리진 데이터 레이어 개수 5 > 3개로 줄이기

In [4]:
import numpy as np

arr = np.loadtxt("cylinder_prediction.xyzc")   # (N,4)
xyz  = arr[:, :3]
lab  = arr[:, 3]

# 1) 잘못된 라벨(2,4) 제거
lab[lab == 4] = 2

# 2) 라벨 3 → 2  (최종 0·1·2)
lab[lab == 3] = 2

arr_fixed = np.hstack([xyz, lab[:, None]])

out_path = "./data/cylinder_pred_3cls.xyzc"
np.savetxt(out_path,
           arr_fixed,
           fmt="%.6e %.6e %.6e %.0e")   # 4열 모두 e‑표기
print("✔ 저장 완료 →", out_path)


✔ 저장 완료 → ./data/cylinder_pred_3cls.xyzc


셀 ② : 데이터로더 정의

In [5]:
import numpy as np, torch, open3d as o3d
from torch.utils.data import Dataset, DataLoader

class XYzcDataset(Dataset):
    """ x y z label (ASCII)  →  중심화·스케일·랜덤샘플링 후 Tensor 반환 """
    def __init__(self, file_list, n_points=4096):
        self.files, self.n_points = file_list, n_points

    def __len__(self):  return len(self.files)

    def __getitem__(self, idx):
        xyzc = np.loadtxt(self.files[idx])           # (N,4)
        xyz, label = xyzc[:, :3], xyzc[:, 3].astype(np.int64)

        # 랜덤 4096 점 샘플 / 부족 시 중복
        sel = np.random.choice(len(xyz), self.n_points, replace=len(xyz)<self.n_points)
        xyz, label = xyz[sel], label[sel]

        # 중심화 + 크기 정규화
        xyz -= xyz.mean(0, keepdims=True)
        xyz /= np.max(np.linalg.norm(xyz, axis=1))

        # Tensor (B,N,3) → (B,3,N) 전치는 학습 셀에서
        return torch.from_numpy(xyz).float(), torch.from_numpy(label)


셀 ③ : 3‑클래스 PointNet 모델

In [6]:
import torch.nn as nn, torch.nn.functional as F

class PointNetSeg(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.bn1, self.bn2, self.bn3 = nn.BatchNorm1d(64), nn.BatchNorm1d(128), nn.BatchNorm1d(1024)

        self.conv4 = nn.Conv1d(1088, 512, 1)
        self.conv5 = nn.Conv1d(512, 256, 1)
        self.conv6 = nn.Conv1d(256, 128, 1)
        self.conv7 = nn.Conv1d(128, num_classes, 1)
        self.bn4, self.bn5, self.bn6 = nn.BatchNorm1d(512), nn.BatchNorm1d(256), nn.BatchNorm1d(128)

    def forward(self, x):               # x:(B,3,N)
        B, _, N = x.size()
        x1 = F.relu(self.bn1(self.conv1(x)))
        x2 = F.relu(self.bn2(self.conv2(x1)))
        x3 = F.relu(self.bn3(self.conv3(x2)))        # (B,1024,N)
        global_feat = torch.max(x3, 2, keepdim=True)[0].repeat(1,1,N)
        x = torch.cat([x1, global_feat], 1)          # (B,1088,N)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.conv7(x)                            # (B,3,N)
        return x


라벨 확인용 점검 코드

In [7]:
import os, numpy as np, torch
from torch.utils.data import DataLoader

# ---------- 경로 ----------
root = "./"           # main.ipynb 폴더
train_files = [
    os.path.join(root, "data/cylinder_synthetic_3cls.xyzc"),
    os.path.join(root, "data/cylinder_pred_3cls.xyzc")
]
val_files = train_files[:]   # 검증용

# ---------- DataLoader (라벨 확인용) ----------
train_loader = DataLoader(
    XYzcDataset(train_files, n_points=4096),
    batch_size=1, shuffle=True, drop_last=False
)

print("=== (A) 첫 배치 라벨 확인 ===")
xyz, label = next(iter(train_loader))
print("unique labels in first batch ->", torch.unique(label))

print("\n=== (B) 파일별 라벨 전수 확인 ===")
for f in train_files + val_files:
    arr  = np.loadtxt(f)          # (N,4)
    labs = arr[:, 3]
    uniq = np.unique(labs)
    fname = os.path.basename(f)
    print(f"{fname:30s} unique labels ->", uniq)

    # --- 라벨 3 또는 0·1·2 이외 값이 있으면 샘플 5개 출력 ---
    bad_mask = ~np.isin(labs, [0, 1, 2])
    if bad_mask.any():
        bad_vals = np.unique(labs[bad_mask])
        print(f"  ⚠️  Found unexpected labels {bad_vals}. Showing first 5 rows:")
        print(arr[bad_mask][:5])      # x y z label 미리보기


=== (A) 첫 배치 라벨 확인 ===
unique labels in first batch -> tensor([0, 1, 2])

=== (B) 파일별 라벨 전수 확인 ===
cylinder_synthetic_3cls.xyzc   unique labels -> [0. 1. 2.]
cylinder_pred_3cls.xyzc        unique labels -> [0. 1. 2.]
cylinder_synthetic_3cls.xyzc   unique labels -> [0. 1. 2.]
cylinder_pred_3cls.xyzc        unique labels -> [0. 1. 2.]


셀 ④ : 학습 설정 & 훈련

In [8]:
from tqdm import trange
import torch, math

device = torch.device("cuda")
model  = PointNetSeg(num_classes=3).to(device)

# ----- 클래스 불균형 가중치 -----
# ----- 클래스 불균형 가중치 (0 division 방지) -----
cnts = np.bincount(
    np.concatenate([np.loadtxt(f)[:,3].astype(int) for f in train_files]),
    minlength=3
).astype(float)

# 0개 클래스 → 가중치 0 (또는 1) 로 설정
cnts[cnts == 0] = np.inf        # 1/inf = 0  → Loss에서 해당 클래스 무시
weights = torch.tensor(1.0 / cnts, dtype=torch.float32).to(device)

criterion = torch.nn.CrossEntropyLoss(weight=weights)
print("class counts :", cnts)
print("loss weights :", weights.cpu().numpy())


# ----- DataLoader -----
train_loader = DataLoader(XYzcDataset(train_files), batch_size=2,
                          shuffle=True, drop_last=False)
val_loader   = DataLoader(XYzcDataset(train_files), batch_size=2,
                          shuffle=False, drop_last=False)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in trange(100, desc="Epoch"):
    model.train(); running=0
    for xyz, label in train_loader:
        xyz, label = xyz.to(device), label.to(device)
        xyz = xyz.permute(0,2,1)
        opt.zero_grad()
        loss = criterion(model(xyz), label)
        loss.backward(); opt.step()
        running += loss.item()
    print(f"  train loss {running/len(train_loader):.4f}")

    # ---- 간단 검증 ----
    model.eval(); correct=tot=0
    with torch.no_grad():
        for xyz, label in val_loader:
            xyz, label = xyz.to(device), label.to(device)
            pred = model(xyz.permute(0,2,1)).argmax(1)
            correct += (pred==label).sum().item(); tot += label.numel()
    print(f"  val acc {correct/tot:.4f}")

torch.save(model.state_dict(), "seg_cylinder_3cls.pth")
print("✅  seg_cylinder_3cls.pth 저장")


class counts : [ 3532.  3402. 23274.]
loss weights : [2.8312570e-04 2.9394473e-04 4.2966400e-05]


Epoch:   3%|▎         | 3/100 [00:00<00:20,  4.75it/s]

  train loss 1.0867
  val acc 0.1150
  train loss 0.8303
  val acc 0.5026
  train loss 0.5490
  val acc 0.7640


Epoch:   5%|▌         | 5/100 [00:00<00:13,  7.19it/s]

  train loss 0.4730
  val acc 0.7699
  train loss 0.4300
  val acc 0.7606
  train loss 0.3981
  val acc 0.7690
  train loss 0.3721


...

  val acc 0.9080
  train loss 0.0787
  val acc 0.9183
  train loss 0.0558
  val acc 0.8671


Epoch:  97%|█████████▋| 97/100 [00:07<00:00, 12.63it/s]

  train loss 0.0696
  val acc 0.7808
  train loss 0.0615
  val acc 0.7421
  train loss 0.0659


Epoch: 100%|██████████| 100/100 [00:07<00:00, 12.70it/s]

  val acc 0.7079
  train loss 0.0542
  val acc 0.6987
  train loss 0.0508
  val acc 0.6630
✅  seg_cylinder_3cls.pth 저장


셀 ⑤ : 원본 스캔 cylinder.xyz 추론 & 저장

In [9]:
import open3d as o3d

# ----- 모델 로드 -----
model.load_state_dict(torch.load("seg_cylinder_3cls.pth", map_location=device))
model.eval()

# ----- 입력 전처리 -----
xyz_raw = np.loadtxt("cylinder.xyz")[:, :3]
xyz_norm = xyz_raw - xyz_raw.mean(0, keepdims=True)
xyz_norm /= np.max(np.linalg.norm(xyz_norm, axis=1))

with torch.no_grad():
    pts = torch.from_numpy(xyz_norm).float().unsqueeze(0).to(device)  # (1,N,3)
    pred = model(pts.permute(0,2,1)).argmax(1).cpu().numpy()[0]

# ----- 저장 -----
xyzc_out = np.hstack([xyz_raw, pred[:,None]])
np.savetxt("cylinder_pred_new.xyzc", xyzc_out, fmt="%.6f %.6f %.6f %d")
print("📝  cylinder_pred_new.xyzc 저장 (라벨 0·1·2)")

pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(xyz_raw)
colors = np.array([[1,0,0],[0,1,0],[0,0,1]])
pc.colors = o3d.utility.Vector3dVector(colors[pred])
o3d.io.write_point_cloud("cylinder_pred_new.ply", pc)
print("📝  cylinder_pred_new.ply 저장 (색으로 확인)")


📝  cylinder_pred_new.xyzc 저장 (라벨 0·1·2)
📝  cylinder_pred_new.ply 저장 (색으로 확인)
